In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import urllib

In [2]:
# url="http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_1"


In [354]:
def getInfo(url,grp_name):
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')

    #!!!!!!!!!!!!!!!!!!!
    region = grp_name
#     region='муниципальный'
    #!!!!!!!!!!!!!!!!!!!
    str_url = checkEnd(url)

#     the_string = url.rsplit('/',1)[1]
#     srch = re.search('[а-яА-Я]', the_string) 
#     if srch == None:
#         str_url = urllib.parse.unquote(the_string)
#     else:
#         str_url = the_string
#     #
    lst = str_url.split("_")
    # 2 - номер маршрута
    nmbr = lst[-1]
    # 1 - тип транспорта
    tp_ts = lst[-2]
    # rt_nmbr = url.rsplit("_",1)[1]

    def getTable(table_rows):
        l = []
        for tr in table_rows:
            try:
                td = tr.find_all('td')
                row = [tr.text for tr in td]
                l.append(row)
            except:
                pass
        #
        return l
    #
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    all_tables = bs.find_all("table")
    l2 = getTable(all_tables[2])
    l3 = getTable(all_tables[3])
    l5 = getTable(all_tables[5])
    l6 = getTable(all_tables[6])
    l7 = getTable(all_tables[7])
    l8 = getTable(all_tables[8])

    ####
    # 3 - наименование маршрута
    str1 = l2[0][0]
    str1 = str1.replace("\n\n","\n")
    rt_nm = str1.split("\n")[1][2:].split("№")[1]
    rt_nm = rt_nm.split('"', 1)[1][:-1]

    ####
    # 4 -маршрут следования
    route_frw = l3[0][1]
    route_bckw = l3[1][1]
    ####

    ####
    # 5,6 - список оп
    def stops(l):
    #
        str_all=''
        for row in l[2:]:
            rw = []
            for i in row:
                i = i.replace("\n","")
                rw.append(i)
            str1 = rw[1]
            str_all = str_all + "_" + str1
        #
        str_all = str_all[1:]
        return str_all
    # 
    stops_frw = stops(l5)
    try:
        stops_bckw = stops(l6)
    except:
        stops_bckw = 'кольцевой'
    ####

    ####
    # 7) Режим работы
    # 8) Интервалы движения
    # 9) Выпуск на линию
    # 10) Время в пути
    # 11) Протяжённость маршрута
    # 12) Обслуживающие предприятия
    # 13) Подвижной состав
    # 14) Стоимость проезда
    # 15) Место отстоя 

    nlst = []
    for row in l7:
        tmplst = []
        for j in row:
            j = j.replace("\n","")
            tmplst.append(j)
        nlst.append(tmplst)
    # 

    work_time = nlst[0][1]
    intervs = nlst[1][1]
    amnt_mch = nlst[2][1]
    duration = nlst[3][1]
    length = nlst[4][1]
    orgnztn = nlst[5][1]
    podv_sost = nlst[6][1]
    payment = nlst[7][1]
    place_rest = nlst[8][1]
    ####


    # get all to list
    lst_all = [tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,
                    work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
    #
    return lst_all

In [156]:
# lst1 = getInfo(url)

In [98]:
# reg = re.compile('[^0-9-]')
# one_line = df['data'][1].split(" мин")[0]
# if "раб" in one_line:
#     one_line2 = one_line.split(":")[1]
#     one_line2 = reg.sub('', one_line2)
#     print(one_line2)

8-20


In [279]:
def checkGrpNm(tmp_grp_nm):
    if tmp_grp_nm == 'маршруты':
        grp_name = 'муниципальный'
    elif  tmp_grp_nm == 'коммерческий_маршруты':
        grp_name = 'коммерческий'
    elif  tmp_grp_nm == 'пригородный_расписания':
        grp_name = 'пригородный'
    elif  tmp_grp_nm == 'сезонный_маршруты':
        grp_name = 'сезонный'
    elif  tmp_grp_nm == 'междугородный_расписания':
        grp_name = 'междугородний'
    else:
        grp_name = 'другой'
    # 
    return grp_name

In [280]:
def checkEnd(url):
    the_string = url.rsplit('/',1)[1]
    srch = re.search('[а-яА-Я]', the_string) 
    if srch == None:
        str_url = urllib.parse.unquote(the_string)
    else:
        str_url = the_string
    #
    return str_url

In [281]:
def linksCity(city):
    lst_links_city = []
    lst_types = ['автобус', 'троллейбус', 'трамвай']
    lst_route_type = ['маршруты','коммерческий_маршруты', 
                  'пригородный_расписания','сезонный_маршруты','междугородный_расписания']
    #
    city_link = 'http://www.samaratrans.info/wiki/index.php/' + city
    i=0
    for i in range(len(lst_types)):
        tp_link = city_link + '_' + lst_types[i] + '_' + lst_route_type[0]
        lst_links_city.append(tp_link)
    #
    bus_link = city_link + '_' + lst_types[0]
    
    i=0
    for i in range(1,len(lst_route_type)):
        link = bus_link + '_' + lst_route_type[i]
        lst_links_city.append(link)
    #
    return lst_links_city

In [282]:
def routeLinks(url_grp):
    links=[]
    stfa = soup.find_all('table')
    for link in stfa[2].find_all('b'):
        try:
            lsh = 'http://www.samaratrans.info' + link.find_all('a')[0].get("href")
            links.append(lsh)
        except:
            pass
    # 
    return links

In [298]:
def getCityData(city):
    lst_links = linksCity(city)
    all_group=[]
    for grp in range(len(lst_links)):
        tmp_grp_nm = checkEnd(lst_links[grp]).split("_",2)[-1]
        grp_name = checkGrpNm(tmp_grp_nm)
        
        lst_group = routeLinks(lst_links[grp])
        all_routes=[]
        for rtl in range(len(lst_group)):
            try:
                one_route = getInfo(lst_group[rtl],grp_name)
            except:
                one_route = [lst_group[rtl]] * 18
            all_routes.append(one_route)
        all_group.append(all_routes)
    return all_group
# 

In [ ]:
# 18

In [284]:
lst_ct = ['Самара', 'Тольятти', 'Сызрань', 'Новокуйбышевск', 
          'Чапаевск', 'Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']

In [ ]:
# lst_big = []
# for i in range(len(lst_ct)):
#     lst = getCityData(lst_ct[i])
#     lst_big.append(lst)

In [285]:
# import time

In [ ]:
%time lst = getCityData(lst_ct[0])

In [ ]:
clmn = []
for i in range(1,len(lst[0])+1):
    str1 = str(i)+'str'
    clmn.append(str1)
# 

In [ ]:
df = pd.DataFrame(data=lst,columns=clmn)

In [ ]:
not_ok = df[df["1str"].str.contains("www", na=False)].reset_index(drop=True)

In [ ]:
print(len(not_ok))

In [353]:
print(not_ok["1str"][0])

http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_7
